# AVISO Data Processing - Converting file format to netcdf

**Purpose**: Code for converting the mat files into netcdf format

**Luke Colosi | lcolosi@ucsd.edu**

Import libraries

In [12]:
import xarray as xr
import h5py
import numpy as np
import os

Set the path to aviso data set

In [13]:
PATH = '/Users/lukecolosi/Desktop/projects/graduate_research/Gille_lab/AirSeaCoupling/data/aviso/'
file_ssh = 'aviso_tot_MADT_nrt_ccs.mat' # Sea Surface Height
file_sla = 'aviso_tot_MSLA_nrt_ccs.mat' # Mean Sea Level Anomaly 

Convert .mat file into a netcdf using xarray

In [14]:
# Open ssh and sla files 
fssh = h5py.File(PATH + file_ssh, "r")
fsla = h5py.File(PATH + file_sla, "r")

# Grab variables names
vars_ssh = [key for key in fssh.keys()]
vars_sla = [key for key in fsla.keys()]

# # Assign variables

#--- SSH ---# 
ssh      = fssh[vars_ssh[0]][()]
time_ssh = fssh[vars_ssh[1]][()]
lon_ssh  = fssh[vars_ssh[2]][()]
lat_ssh  = fssh[vars_ssh[3]][()]

#--- SLA ---# 
sla      = fsla[vars_sla[0]][()]
time_sla = fsla[vars_sla[1]][()]
lon_sla  = fsla[vars_sla[2]][()]
lat_sla  = fsla[vars_sla[3]][()]

# Save variables in data arrays

#--- SSH ---# 
SSH = xr.DataArray(data=np.array(ssh), 
                    dims=['lon','lat','time'],
                    coords=dict(
                        lon=np.squeeze(lon_ssh),
                        lat=np.squeeze(lat_ssh),
                        time=np.squeeze(time_ssh),
                        ),
                    attrs=dict(
                        description='Mean sea surface height from AVSIO Satellite Product.',
                        units='m'
                        )
)

#--- SLA ---# 
SLA = xr.DataArray(data=np.array(sla), 
                       dims=['lon','lat','time'],
                       coords=dict(
                        lon=np.squeeze(lon_sla),
                        lat=np.squeeze(lat_sla),
                        time=np.squeeze(time_sla),
                        ),
                       attrs=dict(
                           description='Mean sea level anomaly from AVSIO Satellite Product.',
                           units='m'
                    )
)

# Create a data set from data arrays 
data = xr.Dataset({'SSH':SSH, 'SLA':SLA})

# Check if file exists, then delete it
file_path = PATH + "aviso_tot_nrt_ccs.nc"
if os.path.exists(file_path):
    os.remove(file_path)

# Create netcdf file
data.to_netcdf(file_path,mode='w')
